# Compilador R Markdown para Notebooks de Jupyter

Creado el **29 de febrero del 2024**
### *Luis Muñiz Valledcor*

In [ ]:
import os, re, subprocess
from google.colab import drive
from ipywidgets import Checkbox, HTML, VBox, HBox, Button
from IPython.display import display
import time

#@markdown ## Preparación del compilador
#@markdown Al ejecutar esta celda, se instarán todos los paquetes, dependencias y definiciones necesarias, para compilar notebooks jupyter en **Google Colaboratory** mediante el renderizador que emplea **R** para construir documentos **R Markdown**.

#@markdown Los paquetes básicos de **R** necesarios para procesar documentos **R Markdwon** se instalan por defecto. Adicionalmente, puede especificar otros paquetes de **R** empleados en su documento.

#@markdown Especifique los nombres de los paquetes requeridos separados por coma.

dir_proyecto = "/content/drive/Shareddrives/Categóricos/QuickRMD_CoLab" #@param {type:"string"}
paquetes = "reticulate" #@param {type:"string"}

print("\n==================================================")
print("CONECTAR CON G-DRIVE")
print("===================================================\n")

drive.mount('/content/drive')

print("\n==================================================")
print("DEFINIR FUNCIONES DE LA CAPA DE COMPATIBILIDAD")
print("===================================================\n")

def instalar_paquetes_r(dir_lib, lista_paquetes):
    """
    Instala los paquetes de R en un directorio específico si no están ya instalados.

    :param dir_lib: La ruta del directorio donde se deben instalar los paquetes de R.
    :param lista_paquetes: Una lista de nombres de paquetes de R para instalar.
    """
    for paquete in lista_paquetes:
        paquete_instalado = False
        for item in os.listdir(dir_lib):
            if os.path.isdir(os.path.join(dir_lib, item)) and item == paquete:
                paquete_instalado = True
                break
        if not paquete_instalado:
            !R -e "install.packages('{paquete}', lib='{dir_lib}')"

    os.environ["R_LIBS_USER"] = dir_lib

def generar_yaml(paquetes_latex, latex_engine='xelatex', wrap='72', es_personalizado=False):
    """
    Genera el YAML para un documento R Markdown.

    Esta función permite especificar paquetes LaTeX que se incluirán en el preámbulo del documento y configura el motor de LaTeX y la opción de ajuste de texto (wrap) en el editor.

    Parámetros:
    - paquetes_latex (str): Una cadena de nombres de paquetes LaTeX separados por comas.
    - latex_engine (str, opcional): El motor de LaTeX a utilizar. Valor predeterminado: 'xelatex'.
    - wrap (str, opcional): El número de columnas para el ajuste de texto en el editor. Valor predeterminado: '72'.

    Retorna:
    - str: El YAML configurado como una cadena de texto.
    """
    # Divide la cadena de paquetes en una lista
    lista_paquetes = paquetes_latex.split(',')

    # Inicializa la cadena de header-includes
    header_includes = ""
    # Agrega cada paquete a la cadena de header-includes
    for paquete in lista_paquetes:
        header_includes += f"- \\usepackage{{{paquete.strip()}}}\n"
    # Añade el path para las imágenes
    if es_personalizado:
        header_includes += f"- \\graphicspath{{"+"{../../img/}"+"}\n" #Las compilaciones personalizadas están en un nivel más abajo
    else:
        header_includes += f"- \\graphicspath{{"+"{../img/}"+"}\n"
    # Construye el YAML completo
    yaml_front_matter = f"""
header-includes:
{header_includes}
output:
  pdf_document:
    latex_engine: {latex_engine}
    fig_caption: true
editor_options:
  markdown:
    wrap: {wrap}
"""
    return yaml_front_matter

def reemplazar_yaml(rmd_contenido, nuevo_yaml):
    """
    Esta función reemplaza el bloque YAML en un documento R Markdown sin eliminar
    las líneas delimitadoras '---'.

    :param rmd_contenido: El contenido del documento R Markdown.
    :param nuevo_yaml: El nuevo bloque YAML que se desea establecer.
    :return: El contenido actualizado del documento R Markdown.
    """
    # Divide el contenido en líneas
    lineas = rmd_contenido.split('\n')

    # Busca las líneas que contienen solo '---' que delimitan el YAML
    indices_delimitadores = [i for i, linea in enumerate(lineas) if linea.strip() == '---']

    # Verifica que se encontraron al menos dos delimitadores para un bloque YAML válido
    if len(indices_delimitadores) >= 2:
        # Reemplaza el contenido entre los delimitadores con el nuevo YAML
        lineas[indices_delimitadores[0] + 1:indices_delimitadores[1]] = nuevo_yaml.strip().split('\n')

    # Reconstruye y devuelve el contenido actualizado
    return '\n'.join(lineas)

def eliminar_yaml(rmd_contenido):
    """
    Esta función elimina el bloque YAML de un documento R Markdown, incluyendo
    las líneas delimitadoras '---'.

    :param rmd_contenido: El contenido del documento R Markdown.
    :return: El contenido actualizado del documento R Markdown sin el bloque YAML.
    """
    # Divide el contenido en líneas
    lineas = rmd_contenido.split('\n')

    # Busca las líneas que contienen solo '---' que delimitan el YAML
    indices_delimitadores = [i for i, linea in enumerate(lineas) if linea.strip() == '---']

    # Verifica que se encontraron al menos dos delimitadores para un bloque YAML válido
    if len(indices_delimitadores) >= 2:
        # Elimina el contenido entre los delimitadores, incluyendo los delimitadores mismos
        del lineas[indices_delimitadores[0]:indices_delimitadores[1] + 1]

    # Reconstruye y devuelve el contenido actualizado sin el bloque YAML
    return '\n'.join(lineas)

def preparar_rmd(ruta_rmd_lectura, ruta_rmd_escritura, paquetes_latex, latex_engine, wrap, es_convertido, eliminar=False, es_personalizado=False):
    """
    Esta función lee un archivo R Markdown, y dependiendo del valor del parámetro 'eliminar',
    modifica su bloque YAML o lo elimina por completo. Luego, guarda el contenido resultante en un nuevo archivo.

    :param ruta_rmd_lectura: El path del archivo R Markdown original para leer.
    :param ruta_rmd_escritura: El path del archivo R Markdown donde se guardará el contenido modificado.
    :param paquetes_latex: Lista de paquetes LaTeX que deben incluirse en el YAML. Solo necesario si eliminar=False.
    :param latex_engine: Motor de LaTeX a utilizar. Solo necesario si eliminar=False.
    :param wrap: Configuración de ajuste para el código en el YAML. Solo necesario si eliminar=False.
    :param eliminar: Booleano que indica si se debe eliminar el YAML (True) o reemplazarlo (False).
    """

    # Leer el contenido del archivo R Markdown original
    with open(ruta_rmd_lectura, 'r') as archivo:
        contenido_rmd = archivo.read()

    # Decidir entre reemplazar o eliminar el bloque YAML
    if eliminar:
        contenido_rmd = eliminar_yaml(contenido_rmd)
    else:
        nuevo_yaml = generar_yaml(paquetes_latex, latex_engine, wrap, es_personalizado)
        contenido_rmd = reemplazar_yaml(contenido_rmd, nuevo_yaml)

    # Ajustar los chunks de Python a R si proviene de un notebook o eliminar el chunk setup si proviene de un Rmd
    if es_convertido:
        rmd_limpio = ajustar_chunks_python_a_r(contenido_rmd)
    else:
        rmd_limpio = eliminar_chunk_setup(contenido_rmd)

    # Escribir el contenido modificado en el nuevo archivo R Markdown
    with open(ruta_rmd_escritura, 'w') as archivo_nuevo:
        archivo_nuevo.write(rmd_limpio)

def ajustar_chunks_python_a_r(contenido):
    """
    Ajusta los chunks de Python en un documento R Markdown para que sean compatibles con R,
    identificando y reemplazando un comentario especial en la primera línea del chunk. Además,
    verifica que haya contenido válido dentro de las llaves seguido de #ck.

    Parámetros:
    - contenido: El contenido del documento R Markdown.

    Retorna:
    - El contenido del documento con los chunks ajustados.
    """
    # Procesa cada línea del contenido para ajustar los chunks
    lineas = contenido.split('\n')
    lineas_ajustadas = []
    i = 0

    while i < len(lineas):
        linea = lineas[i]#.strip()

        # Omitir líneas que comienzan con los patrones de región específicos
        if linea.startswith("<!-- #region") or linea.startswith("<!-- #endregion"):
            # Simplemente incrementar el índice para no incluir esta línea en el resultado
            i += 1
            continue

        # Encuentra el inicio de un chunk de Python
        if linea.startswith("```{python"):
            i += 1  # Avanza a la siguiente línea para leer el comentario especial
            if i < len(lineas):
                primera_linea = lineas[i].replace(" ", "")
                # Busca el comentario especial y verifica contenido entre llaves
                if primera_linea.startswith("#ck{") and primera_linea.endswith("}") and len(primera_linea) > 5:
                    # Extrade el contenido entre llaves (los parámetros del chunk)
                    patron=re.compile(r'(\{.*?\})') #Subcadena con los parámetros
                    opciones_chunk = patron.search(lineas[i]).group(0)
                    # Añade la línea limpia como inicio del chunk de R
                    lineas_ajustadas.append(f"```{opciones_chunk}")
                else:
                    # Si no encuentra el comentario especial con contenido válido, mantiene el chunk como de R estándar
                    lineas_ajustadas.append("```{r}")
                    i -= 1  # Vuelve a revisar la misma línea en la siguiente iteración si no se encontró el comentario especial válido.
        else:
            # Añade la línea sin cambios si no es el inicio de un chunk
            lineas_ajustadas.append(linea)
        i += 1

    return '\n'.join(lineas_ajustadas)

def eliminar_chunk_setup(contenido):
    """
    Elimina el chunk de configuración 'setup' de un documento RMarkdown.

    Parámetros:
    - contenido: El contenido del documento RMarkdown como una cadena de texto.

    Retorna:
    - El contenido del documento RMarkdown modificado, sin el chunk de configuración.
    """
    # Dividir el contenido en líneas para procesarlo
    lines = contenido.split('\n')
    inside_setup_chunk = False
    filtered_lines = []

    for line in lines:
        # Detectar el inicio del chunk de configuración
        if line.startswith('```{r setup'):
            inside_setup_chunk = True
            continue  # No añadir esta línea ni las siguientes del chunk al resultado final

        # Detectar el final del chunk
        if inside_setup_chunk and line.startswith('```'):
            inside_setup_chunk = False
            continue  # No añadir la línea de cierre del chunk al resultado final

        # Añadir líneas que no están dentro del chunk de configuración
        if not inside_setup_chunk:
            filtered_lines.append(line)

    # Unir las líneas procesadas en una sola cadena de texto
    return '\n'.join(filtered_lines)

def preparar_documentos(notebooks_seleccionados, dir_build_jupytext, dir_prep_build_rmarkdown, paquetes_latex, latex_engine, wrap, es_personalizado=False):
    """
    Procesa una lista de notebooks de Python y archivos RMarkdown para su conversión y adecuación.

    Parámetros:
    - notebooks_seleccionados: Lista de tuplas (ruta del archivo, nombre del archivo).
    - dir_build_jupytext: Directorio donde se guardarán los Rmd convertidos.
    - dir_prep_build_rmarkdown: Directorio donde se guardarán los Rmd preparados.
    - paquetes_latex: Paquetes LaTeX requeridos.
    - latex_engine: Motor LaTeX a usar.
    - wrap: Configuración de ajuste de texto.
    - es_personalizado: Indica si se deben conservar o no los bloques YAML personalizados.
    """
    conservar_yalm = True
    rmarkdowns = []

    for arch_path, file in notebooks_seleccionados:
        original_rmd_path = os.path.join(dir_build_jupytext, file.replace(".ipynb", ".Rmd"))
        prep_bild_rmd_path = os.path.join(dir_prep_build_rmarkdown, os.path.basename(original_rmd_path))

        es_convertido = arch_path.endswith('.ipynb')  # Indica si el archivo es un notebook de Python

        # Convertir notebook a RMarkdown si es necesario
        if es_convertido:
            subprocess.run(['jupytext', '--to', 'rmarkdown', '--output', original_rmd_path, arch_path])
        else:
            subprocess.run(['cp', arch_path, original_rmd_path])

        # Realizar las adecuaciones necesarias en los archivos
        if conservar_yalm:
            preparar_rmd(original_rmd_path, prep_bild_rmd_path, paquetes_latex, latex_engine, wrap, es_convertido, eliminar=False, es_personalizado=es_personalizado)
            conservar_yalm = False
        else:
            preparar_rmd(original_rmd_path, prep_bild_rmd_path, paquetes_latex, latex_engine, wrap, es_convertido, eliminar=True)

        rmarkdowns.append(prep_bild_rmd_path)

    return rmarkdowns

def integrar_documentos_rmd(ruta_documento_Rmd_final, rmarkdowns):
    """
    Integra múltiples documentos RMarkdown en un único documento.

    Parámetros:
    - ruta_documento_Rmd_final: Ruta al archivo RMarkdown final.
    - rmarkdowns: Lista de rutas a los documentos RMarkdown a integrar.
    """
    # Abrir el archivo de salida en modo de escritura
    with open(ruta_documento_Rmd_final, 'w') as salida:
        # Iterar sobre la lista de archivos de entrada
        for ruta in rmarkdowns:
            # Asegurarse de que el archivo existe
            try:
                # Abrir el archivo actual en modo de lectura
                with open(ruta, 'r') as entrada:
                    # Leer el contenido del archivo y escribirlo en el archivo de salida
                    salida.write(entrada.read())
                    # Añadir un salto de línea entre contenidos de archivos diferentes
                    salida.write('\n')
            except FileNotFoundError:
                print(f"El archivo {ruta} no existe y será omitido.")

def listar_notebooks_rmd(directory):
    """
    Recopila y ordena alfabéticamente todos los archivos .ipynb y .Rmd en un directorio dado.

    Parámetros:
    - directory: Ruta del directorio donde buscar los archivos.

    Retorna:
    - Una lista de tuplas, cada tupla contiene la ruta completa y el nombre de un archivo.
    """
    archivos = []  # Lista para almacenar las rutas y nombres de los archivos

    # Recorrer todos los archivos .ipynb y .Rmd en el directorio
    for root, dirs, files in os.walk(directory):
        for file in sorted(files):
            if file.endswith(".ipynb") or file.endswith(".Rmd"):
                file_path = os.path.join(root, file)
                archivos.append((file_path, file))

    # Ordenar la lista completa de archivos por nombre de archivo
    archivos.sort(key=lambda x: x[1])

    return archivos

print("\n==================================================")
print("INSTALAR HERRAMIENTAS Y DEPENDENCIAS EN EL SISTEMA")
print("===================================================\n")

!apt-get update
!apt-get install texlive-xetex texlive-fonts-recommended texlive-latex-extra
!apt-get install pandoc
!pip install jupyter nbconvert
!pip install jupytext

print("\n==================================================")
print("INSTALAR PAQUETES DE R")
print("===================================================\n")

lista_paquetes = [paquete.strip() for paquete in paquetes.split(',')]
dir_lib = dir_proyecto + "/lib"

instalar_paquetes_r(dir_lib, lista_paquetes)

print("\n==================================================")
print("EL COMPILADOR ESTA LISTO")
print("====================================================\n")

In [ ]:
#@markdown ## Compilar Notebook final
#@markdown En esta celda se compilan todos los Notebooks del proyecto. El documento resultante se crea en el subdirectorio `build` del proyecto con el nombre especificado en el campo `nombre_documento`.
#@markdown  Para que la compilación sea efectiva, se requiere que todos los notebooks estén depurados y sólo se recomienda realizar la compilación cuando el documento sea estable en su totalidad.
#@markdown  Para etapas preliminares (cuando hay varios miembros del equipo editando simultáneamente sus notebooks de trabajo) se recomienda una compilación personalizada.
#@markdown Eso permite crear compilaciones sólo con los notebooks que interesan a cada miembro del equipo.

#@markdown ### Ruta del directorio de trabajo

#@markdown ### Nombre del documento final (nombre individual)
nombre_documento = "Tarea2" #@param {type:"string"}

#@markdown Tiempo de espera antes de iniciar la compilación.
retardo = 0 #@param {type:"integer"}

#@markdown ### Configuración YAML del documento R Markdown
#@markdown Incluye las configuraciones para el encabezado y opciones del documento.
latex_engine = "xelatex" #@param ["pdflatex", "xelatex", "lualatex"]
wrap = 72 #@param {type:"integer"}
paquetes_latex = "mathtools, amsmath" #@param {type:"string"}

dir_notebooks = dir_proyecto + "/notebooks"
dir_build = dir_proyecto + "/build/"
dir_build_jupytext = dir_proyecto + "/build/partial-build/original-jupytext"
dir_prep_build_rmarkdown = dir_proyecto + "/build/partial-build/prep-build-rmarkdown"

ruta_documento_Rmd_final = dir_build + nombre_documento + ".Rmd"

print("\n==================================================")
print("PREPARACIÓN DE LOS DIRECTORIOS PARA LA COMPILACIÓN")
print("====================================================\n")

#!rm -r "{dir_build}"
!mkdir -p "{dir_build}"
!mkdir -p "{dir_build_jupytext}"
!mkdir -p "{dir_prep_build_rmarkdown}"

time.sleep(retardo)

print("\n==================================================")
print("OBTENCIÓN DE TODOS LOS NOTEBOOKS DISPONIBLES")
print("====================================================\n")

# Comando para eliminar archivos específicos en dir_build
!find {dir_build} -name "{nombre_documento}.*" -delete

# Comando para eliminar todos los archivos en dir_build_jupytext
!rm -rf {dir_build_jupytext}/*

# Comando para eliminar todos los archivos en dir_prep_build_rmarkdown
!rm -rf {dir_prep_build_rmarkdown}/*

#conservar_yalm=True
# Lista para almacenar las rutas de los archivos .ipynb
notebooks = []
# Lista para almacenar las rutas de los archivos .Rmd procesados individualmente
#rmarkdowns = []

# Recopilar todos los archivos .ipynb y .Rmd
notebooks = listar_notebooks_rmd(dir_notebooks)

print("\n==================================================")
print("CONVERSIÓN Y PREPARACIÓN PARA LA COMPILACIÓN")
print("====================================================\n")

# Procesar cada archivo .ipynb o .Rmd ordenadamente
rmarkdowns = preparar_documentos(notebooks, dir_build_jupytext, dir_prep_build_rmarkdown, paquetes_latex, latex_engine, wrap)

# Abrir el archivo de salida en modo de escritura
integrar_documentos_rmd(ruta_documento_Rmd_final, rmarkdowns)

print("\n==================================================")
print("COMPILAR R MARKDOWN")
print("====================================================\n")

!R -e "rmarkdown::render('{ruta_documento_Rmd_final}', output_format='pdf_document')"


In [ ]:
#@markdown ## Seleccionar Notebooks para una compilación personalizada
#@markdown En esta celda se seleccionan los Notebooks para construir una compilación parcial del documento.
#@markdown La compilación resultante se almacena en un subdirectorio del mismo nombre especificado en el campo `nombre_documento`.
#@markdown La finalidad de esta sección es crear compilaciones personalizadas e independientes para cada integrante del equipo.
#@markdown Uno de los usos más útiles es la depuración de los notebooks de cada integrante, sin la intromisión de errores introducidos durante la edición de los notebooks por otros integrantes del equipo.

#@markdown ### Nombre del documento personalizado
nombre_documento = "compilación-rik" #@param {type:"string"}

#@markdown Tiempo de espera antes de iniciar la búsqueda.
retardo = 1 #@param {type:"integer"}

#@markdown ### Configuración YAML del documento R Markdown
#@markdown Incluye las configuraciones para el encabezado y opciones del documento.
latex_engine = "xelatex" #@param ["pdflatex", "xelatex", "lualatex"]
wrap = 72 #@param {type:"integer"}
paquetes_latex = "mathtools, amsmath" #@param {type:"string"}

dir_notebooks = dir_proyecto + "/notebooks"
dir_build = dir_proyecto + "/custom-build/" + nombre_documento + "/"
dir_build_jupytext = dir_build + "partial-build/original-jupytext"
dir_prep_build_rmarkdown = dir_build + "partial-build/prep-build-rmarkdown"

ruta_documento_Rmd_final = dir_build + nombre_documento + ".Rmd"

seleccionado_notebook=True

time.sleep(retardo)

# Lista para almacenar las rutas de los archivos .ipynb
notebooks = []

# Crear una lista para guardar los widgets checkbox
checkboxes = []

# Listar y ordenar alfabéticamente los archivos, luego crear un checkbox para cada archivo .ipynb y
notebooks = listar_notebooks_rmd(dir_notebooks)

boton_todos = Button(description='Todos')
boton_ninguno = Button(description='Ninguno')

# Función para seleccionar todos los checkboxes
def seleccionar_todos(b):
    for checkbox in checkboxes:
        checkbox.value = True

# Función para deseleccionar todos los checkboxes
def deseleccionar_todos(b):
    for checkbox in checkboxes:
        checkbox.value = False

# Conexión de botones con sus funciones correspondientes
boton_todos.on_click(seleccionar_todos)
boton_ninguno.on_click(deseleccionar_todos)

# Creación de checkboxes para cada archivo .ipynb
seleccionado_notebook = True
for path, archivo in notebooks:
    checkbox = Checkbox(value=seleccionado_notebook, description=archivo)
    checkboxes.append(checkbox)
    seleccionado_notebook = False  # Después del primero, todos están deseleccionados

# Muestra los controles y las casillas de verificación centradas
controles = HBox([boton_todos, boton_ninguno], layout={'justify_content': 'center'})
lista_checkboxes = VBox(checkboxes, layout={'align_items': 'center'})
display(HTML('<div style="text-align: center;"><h3>Seleccione los notebooks para la construcción del documento:</h3></div>'))
display(VBox([controles, lista_checkboxes]))

In [ ]:
#@markdown ### Compilar Notebook personalizado
#@markdown Tiempo de espera antes de iniciar la compilación personalizada.

retardo = 10 #@param {type:"integer"}

print("\n==================================================")
print("PREPARACIÓN DE LOS DIRECTORIOS PARA LA COMPILACIÓN")
print("====================================================\n")

#!rm -r "{dir_build}"
!mkdir -p "{dir_build}"
!mkdir -p "{dir_build_jupytext}"
!mkdir -p "{dir_prep_build_rmarkdown}"

# Comando para eliminar archivos específicos en dir_build
!find {dir_build} -name "{nombre_documento}.*" -delete

# Comando para eliminar todos los archivos en dir_build_jupytext
!rm -rf {dir_build_jupytext}/*

# Comando para eliminar todos los archivos en dir_prep_build_rmarkdown
!rm -rf {dir_prep_build_rmarkdown}/*

time.sleep(retardo)

print("\n==================================================")
print("OBTENCIÓN DE TODOS LOS NOTEBOOKS SELECCIONADOS")
print("====================================================\n")

#conservar_yalm=True

# Lista para almacenar las rutas de los archivos .Rmd procesados individualmente
#rmarkdowns = []

# Iterar sobre los checkboxes y mostrar los nombres de los archivos seleccionados
archivos_seleccionados = [cb.description for cb in checkboxes if cb.value]

notebooks_seleccionados = [tupla for tupla in notebooks if tupla[1] in archivos_seleccionados]

print("\n==================================================")
print("CONVERSIÓN Y PREPARACIÓN PARA LA COMPILACIÓN")
print("====================================================\n")

# Procesar cada archivo .ipynb ordenadamente
rmarkdowns = preparar_documentos(notebooks_seleccionados, dir_build_jupytext, dir_prep_build_rmarkdown, paquetes_latex, latex_engine, wrap, True)

# Abrir el archivo de salida en modo de escritura
integrar_documentos_rmd(ruta_documento_Rmd_final, rmarkdowns)

print("\n==================================================")
print("COMPILAR R MARKDOWN")
print("====================================================\n")

!R -e "rmarkdown::render('{ruta_documento_Rmd_final}', output_format='pdf_document')"
